# Table of Contents
 <p>

In [1]:
import os 
# data_path = '/home/lfaivish/PycharmProjects/Deepshit/DATA_FOLDER'
os.chdir("C:\\Users\\imazeh\\Itzik\\Health_prof\\git_team\\DataScientists")

from spark import utils_function_spark
from spark import arrange_intervals

import gc
import numpy as np
from pyspark import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql.functions import ceil, unix_timestamp
from pyspark.sql.functions import rank
from pyspark.sql.functions import collect_list, array
from pyspark.mllib.linalg import Vectors, VectorUDT
from pyspark.sql.types import *
import time

from scipy.interpolate import interpolate
import pywt
from future.utils import lmap
import numpy as np
from functools import partial

#ML imports:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
from tsfresh import extract_features
from tsfresh.feature_extraction import feature_calculators
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from Utils.Features import ts_fresh
import pandas as pd

C:\Users\imazeh\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
spark = SparkSession.builder.appName("full_flow").getOrCreate()
# .config("spark.sql.shuffle.partitions", "3")

In [4]:
raw_data = spark.read.format('com.databricks.spark.csv').option("header", "True").option("delimiter", ",")\
                      .load('C:/Users/imazeh/Itzik/Health_prof/L_dopa/Large_data/'+'all_lab_data.csv')

In [5]:
t = time.time()
raw_data = arrange_intervals.casting_data(raw_data)
print('stage1 ' + str(time.time() - t))
data_frame = arrange_intervals.make_inteval_per_task(raw_data)
print('stage2 ' + str(time.time() - t))

data_frame = arrange_intervals.tasks_to_intervals(data_frame,
                                                  slide = 2.5, window = 5)
print('tasks_to_intervals ' + str(time.time() - t))

stage1 0.5450022220611572
stage2 116.92121052742004
tasks_to_intervals 125.86327147483826


In [ ]:
#data_frame = arrange_intervals.get_features(data_frame)
#print('stage4 ' + str(time.time() - t))

In [6]:
gc.collect()

data_frame.show(5)
print('data_frame_show ' + str(time.time() - t))

+--------------------+--------------------+--------------------+--------------------+
|                 key|                   X|                   Y|                   Z|
+--------------------+--------------------+--------------------+--------------------+
|[3.1923658E7,0.0,...|[0.86584275960922...|[-0.0852674022316...|[-0.2327923923730...|
|[3.1923658E7,0.0,...|[0.84788239002227...|[-0.0777988657355...|[-0.3939299285411...|
|[3.1923658E7,0.0,...|[0.60710769891738...|[-0.0039055959787...|[-0.2368490248918...|
|[3.1923658E7,0.0,...|[0.93031895160675...|[-0.1709642708301...|[-0.3897138833999...|
|[3.1923658E7,0.0,...|[0.73621869087219...|[0.46842575073242...|[-0.2064738124608...|
+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

data_frame_show 142.1353325843811


In [8]:
df_sample = data_frame.sample(False, 0.001)

In [85]:
print(df_sample.count())
print(type(df_sample))

44
<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
sub_df = df_sample.take(30)
print(type(sub_df))

In [ ]:
sub_df_x = []
for window in sub_df:
#     print(len(window[1]))
    sub_df_x.append(window[1])

In [94]:
sub_df_x = sub_df[0][1]
print(type(sub_df_x), len(sub_df_x))

<class 'list'> 251


In [ ]:
sub_tsf = ts_fresh.convert_signals_for_ts_fresh(sub_df_x, dimension_name='x')

In [ ]:
sub_tsf_features = extract_features(sub_tsf, default_fc_parameters=MinimalFCParameters(),
                                        column_id="signal_id", column_sort="time")

In [17]:
df_sample.cache()

DataFrame[key: struct<_1:double,_2:double,_3:double,_4:double>, X: array<double>, Y: array<double>, Z: array<double>]

In [ ]:
features_dict = {'abs_energy': None,
                 'absolute_sum_of_changes': None,
                 'approximate_entropy': [{'m': 2, 'r': 0.25},
                                         {'m': 2, 'r': 0.5},
                                         {'m': 2, 'r': 0.75}]
                }

In [7]:
def extract_our_features(signal, features_dict):
    features_names = []
    features_vals = []
    for feature in features_dict:
        if features_dict[feature]==None:
            feat_method = 'feature_calculators.'+feature
            features_names.append(feat_method.split('.')[1])
            feat_val = eval(feat_method)(x = signal)
            features_vals.append(feat_val)
        else:
            for param_comb in features_dict[feature]:
                args = ['x=signal']
                name = [feature]
                for param in param_comb.keys():
                    val = param_comb[param]
                    arg = str(param)+'='+str(val)
                    args.append(arg)
                    name.append(arg)
                features_names.append('_'.join(name))
                args_string = ', '.join(args)
                feat_method = 'feature_calculators.'+feature+'('+args_string+')'
                feat_val = eval(feat_method)
                features_vals.append(feat_val)
    return features_names, features_vals

In [192]:
for i
names, features = extract_our_features(sub_df_x, features_dict=features_dict)

In [193]:
print(names)
print(features)

['abs_energy', 'absolute_sum_of_changes', 'approximate_entropy_m=2_r=0.25', 'approximate_entropy_m=2_r=0.5', 'approximate_entropy_m=2_r=0.75']
[3.2672816213142664, 17.14395582373254, 1.1422319036582578, 1.1115601464756923, 0.8749603393165799]


In [98]:
bla = 'feature_calculators.abs_energy'
eval(bla + '()')([10,1,2,3])

114

In [78]:
EfficientFCParameters()

{'abs_energy': None,
 'absolute_sum_of_changes': None,
 'agg_linear_trend': [{'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'var'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'max'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'min'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'mean'},
  {'attr': 'rvalue', 'chunk_len': 50, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'max'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'min'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'mean'},
  {'attr': 'intercept', 'chunk_len': 5, 'f_agg': 'var'},
  {'attr': 'intercept', 'chunk_len': 

In [59]:
def tsf_features_for_single_window(signal_as_list, dimension_name='x', tsf_features=MinimalFCParameters()):
    signal = [signal_as_list]
    signal_for_tsf = ts_fresh.convert_signals_for_ts_fresh(signal, dimension_name=dimension_name)
    features = extract_features(signal_for_tsf, default_fc_parameters=tsf_features,
                                        column_id="signal_id", column_sort="time")
    feat = features.iloc[0]
    flt_feat = [float(i) for i in feat]
    return flt_feat

In [60]:
x = df_sample.take(1)[0][1]

In [61]:
feat = tsf_features_for_single_window(x)

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.97s/it]


In [62]:
for f in feat:
#     ff = float(f)
    print(type(f))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


In [63]:
schema = StructType([
    StructField("F1", FloatType(), False),
    StructField("F2", FloatType(), False),
    StructField("F3", FloatType(), False),
    StructField("F4", FloatType(), False),
    StructField("F5", FloatType(), False),
    StructField("F6", FloatType(), False),
    StructField("F7", FloatType(), False),
    StructField("F8", FloatType(), False)
])

tsf_features_for_single_window_udf = udf(tsf_features_for_single_window, schema)

In [64]:
data_frame_new = df_sample.withColumn('x_features', tsf_features_for_single_window_udf('X'))

In [65]:
data_frame_new.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 key|                   X|                   Y|                   Z|          x_features|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[4.8335001E7,2.0,...|[-0.1336321979761...|[0.44264653325080...|[0.87043887376785...|[-23.021496,-0.09...|
|[6.2129201E7,0.0,...|[-0.8657528162002...|[0.41265270113945...|[-0.0724230185151...|[-216.77802,-0.83...|
|[6.213751E7,0.0,0...|[-0.0735217779874...|[0.80493259429931...|[0.57482403516769...|[-20.981024,-0.08...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [75]:
data_frame_new.select('key.*','x_features').withColumnRenamed('_1', 'key')\
                                           .withColumnRenamed('_2', 'tremor')\
                                           .withColumnRenamed('_3', 'brady')\
                                           .withColumnRenamed('_4', 'dyskinesia').show(5)

+-----------+------+-----+----------+--------------------+
|        key|tremor|brady|dyskinesia|          x_features|
+-----------+------+-----+----------+--------------------+
|4.8335001E7|   2.0|  1.0|       0.0|[-23.021496,-0.09...|
|6.2129201E7|   0.0|  1.0|       0.0|[-216.77802,-0.83...|
| 6.213751E7|   0.0|  0.0|       0.0|[-20.981024,-0.08...|
|6.9924382E7|   0.0|  1.0|       0.0|[81.5845,0.347206...|
|6.9928378E7|   2.0|  1.0|       0.0|[-89.13931,-0.237...|
+-----------+------+-----+----------+--------------------+
only showing top 5 rows



In [68]:
data_frame_new.cache()

DataFrame[key: struct<_1:double,_2:double,_3:double,_4:double>, X: array<double>, Y: array<double>, Z: array<double>, x_features: struct<F1:float,F2:float,F3:float,F4:float,F5:float,F6:float,F7:float,F8:float>]

In [ ]:
feat = tsf_features_for_single_window(x_signal, dimension_name, tsf_features)

In [ ]:
assembler = VectorAssembler(inputCols=names, outputCol="features")
df = assembler.transform(df)
df = df.select([c for c in df.columns if c not in feature_cols])


# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="tremor", outputCol="indexedLabel").fit(df)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="features", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "tremor", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only
# $example off$

spark.stop()